In [56]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from urllib.parse import urlencode
import datetime

# Load environment variables in a file called .env

load_dotenv(override=True)

api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please get your api key and reference it in .env!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [57]:
openai = OpenAI()

In [7]:
# Sending first message to the OpenAI

message = "Hello, OpenAI GPT! This is my first time connecting to you!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role": "user", "content": message}])

print(response.choices[0].message.content)

Hello! Welcome! I'm glad you're here. How can I assist you today?


In [12]:
# asking some math questions

messages = [
    {"role": "system", "content": "you are a math expert who should help with math questions, with sarcasm"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [13]:
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, wow, what a challenge! Are you ready for this? Drumroll, please... It's 4! Shocking, I know. If only all math was this mind-blowing!


In [14]:
messages = [
    {"role": "system", "content": "you are a java programming expert who solves algorithm questions in java"},
    {"role": "user", "content": "Can you solve prime numbers efficiently?"}
]

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Yes, I can help you efficiently generate and find prime numbers in Java using various algorithms. One of the most efficient methods to find all prime numbers up to a given limit \( n \) is the **Sieve of Eratosthenes** algorithm. This algorithm is particularly efficient for generating a list of primes and works in \( O(n \log(\log(n))) \) time.

### Sieve of Eratosthenes Implementation in Java

Here’s how you can implement the Sieve of Eratosthenes in Java:

```java
import java.util.Arrays;

public class SieveOfEratosthenes {

    public static void main(String[] args) {
        int n = 100; // You can set this to any upper limit
        boolean[] isPrime = sieveOfEratosthenes(n);
        System.out.println("Prime numbers up to " + n + ":");
        for (int i = 2; i <= n; i++) {
            if (isPrime[i]) {
                System.out.print(i + " ");
            }
        }
    }

    public static boolean[] sieveOfEratosthenes(int n) {
        // Initialize a boolean array to track p

In [18]:
# A Function to scrape a website

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [27]:
result = Website("https://www.reddit.com/")
print(result.title)
print(result.text)

Reddit - Dive into anything
Skip to main content
Open menu
Open navigation
Go to Reddit Home
Get App
Get the Reddit app
Log In
Log in to Reddit
Expand user menu
Open settings menu
Lakers-Hornets trade nixed
The Lakers have rescinded their offer for Mark Williams
r/NBATalk
and more
DK Pick6
Download DraftKings Pick6 now!
u/DraftKings
Official
Promoted
7.6 earthquake in Caribbean
Magnitude 7.6 earthquake in the Caribbean triggers a tsunami advisory for Puerto Rico and Virgin Islands
r/news
and more
UFC 312 results
This Is Too Wholesome
r/ufc
and more
Sheeran stopped by police
A police officer pulled the plug when Ed Sheeran surprised everyone on Church Street😂😭😭😭
r/BollyBlindsNGossip
and more
Butler debuts with Warriors
Jimmy Butler's 1st Bucket as a Warrior - A Lob from Buddy Hield
r/warriors
and more
2025 DGA Awards
Demi Moore in Dior at the 77th Annual DGA Awards (February 8, 2025)
r/whatthefrockk
and more
Hot
Open sort options
Best
Hot
New
Top
Rising
Everywhere
Open sort options
Ever

In [45]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n"
    user_prompt += website.text
    return user_prompt

In [50]:
def messages_for(website): 
    prompt = user_prompt_for(website)
    if not prompt:
        raise ValueError("user_prompt_for(website) returned an empty string!")
    
    return [
        {"role": "system", "content": "You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown."},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [51]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [52]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [53]:
display_summary("https://reddit.com")

# Reddit - Dive into Anything

Reddit is a social media platform that allows users to participate in discussions across a variety of topics. It comprises numerous communities, known as subreddits, where users can share content, ask questions, and engage with others.

## Recent Highlights
- **Sports News**: 
  - The **Lakers** have decided to withdraw their trade offer for **Mark Williams**.
  - **Jimmy Butler** debuted with the **Warriors**, marking his first points in the team through a lob assist from **Buddy Hield**.
  
- **Natural Disaster**: 
  - A **7.6 magnitude earthquake** in the **Caribbean** has led to a tsunami advisory for **Puerto Rico** and the **Virgin Islands**.

- **Entertainment News**: 
  - **Ed Sheeran** was humorously interrupted by police while making a surprise appearance on **Church Street**.

## Trending Topics
Users engage in varied discussions, including thoughts on wealth and societal progress, celebrity behavior, and significant current events, encapsulated through various threads in communities like r/AskReddit and r/news.

Overall, Reddit serves as a platform for diverse conversation and community interaction, appealing to a wide audience interested in different subjects.

# Stock Analysis using Yahoo Finance

In [67]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"

    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params = {"frequency": "1wk", "period1": datetime_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        html_table_data = soup.find("table")
        print(html_table_data)

        return title, html_table_data

In [59]:
def build_stock_analysis_prompt(stock_symbol, title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""

    user_prompt = f"The data provided below in the HTML table format for {stock_symbol}: {title} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."

    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [62]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [68]:
display_analysis("GOOG")

<table class="table yf-1jecxey"><thead class="yf-1jecxey"><tr class="yf-1jecxey"><th class="yf-1jecxey">Date  </th><th class="yf-1jecxey">Open  </th><th class="yf-1jecxey">High  </th><th class="yf-1jecxey">Low  </th><th class="yf-1jecxey">Close   <span class="container yf-qvmg5x"><div aria-hidden="true" class="icon fin-icon primary-icn sz-medium tw-align-text-top yf-4e31vg"><!-- HTML_TAG_START --><svg viewbox="0 0 24 24" xmlns="http://www.w3.org/2000/svg"><path d="M11 7h2v2h-2zm0 4h2v6h-2zm1-9C6.48 2 2 6.48 2 12s4.48 10 10 10 10-4.48 10-10S17.52 2 12 2m0 18c-4.41 0-8-3.59-8-8s3.59-8 8-8 8 3.59 8 8-3.59 8-8 8"></path></svg><!-- HTML_TAG_END --></div> <div class="tooltip al-bottom yf-qvmg5x" role="tooltip"><div class="arrow yf-qvmg5x"></div> <h3 class="title yf-qvmg5x"></h3> <span class="toolTipContent yf-1jecxey" slot="content">Close price adjusted for splits. </span></div> </span> </th><th class="yf-1jecxey">Adj Close   <span class="container yf-qvmg5x"><div aria-hidden="true" class="i

# Analysis of Alphabet Inc. (GOOG) Stock Trends

## Overview
The provided data represents the weekly historical prices for Alphabet Inc. (GOOG), covering various key metrics including opening price, high, low, close prices, and trading volume. It appears to cover the year leading up to early February 2025, with fluctuations in stock prices that reflect market trends.

### Key Metrics from the Table:
- **Open:** The price at which the stock opened for trading on that day.
- **High:** The highest price reached during the trading session.
- **Low:** The lowest price during the trading session.
- **Close:** The final price of the stock when the market closed.
- **Adj Close:** The adjusted close price accounting for dividends and stock splits.
- **Volume:** The number of shares traded.

## Summary of Trends

### Price Movement
- **Overall Trend:** The stock price exhibited volatility throughout the period. 
    - The **highest closing price** recorded was **$205.60** on January 27, 2025.
    - The **lowest price** seen was approximately **$136.29** on March 4, 2024.
- **Recent Months (Late 2024 - Early 2025):** The stock seems to have experienced a decline from earlier highs, notably closing at **$187.14** on February 7, 2025 after previously peaking over **$200** just weeks prior.
- **More Historical Context (Early 2024):** Earlier in 2024, the stock dropped to near **$137**, reflecting broader market conditions or company-specific issues.

### Trading Volume
- Trading volumes peaked significantly during particular weeks, notably:
    - **139,005,000 shares** traded on February 3, 2025, indicating high interest or reaction to news/events related to the company.
    - Notably lower volumes can be seen in earlier periods, suggesting fluctuating investor interest at different times.

### Dividends
- There were dividends paid out on **December 9, 2024** and **September 9, 2024**, among other dates, indicating the company's policy of returning cash to shareholders.

### Volatility
- The stock price fluctuates significantly within short periods, with substantial daily price movements suggesting an environment of uncertainty.

## Return Calculation
To determine total returns over this period:
1. **Initial Price:** Starting from the **closing price of $136.29** on **March 4, 2024**.
2. **Final Price:** Ending with the **closing price of $205.60** on **January 27, 2025**.
3. **Calculation of Returns:**
   \[
   \text{Total Return} = \left( \frac{\text{Final Price} - \text{Initial Price}}{\text{Initial Price}} \right) \times 100
   \]
   \[
   \text{Total Return} = \left( \frac{205.60 - 136.29}{136.29} \right) \times 100 = 50.76\%
   \]

### Conclusion
The analysis indicates that Alphabet Inc. (GOOG) has shown considerable price volatility, with significant drops and rebounds over the past year. Despite the fluctuations, an investor could have expected a total return of approximately **50.76%** during this timeframe, which indicates potential for significant capital appreciation in the stock. This highlights the importance of monitoring market conditions and potential internal factors influencing stock performance.

In [64]:
display_analysis("CRM")

## Analysis of Salesforce, Inc. (CRM) Stock Trends

### Overview

The table presents the historical stock prices for Salesforce, Inc. (CRM) for various weeks, including key data points: **Open**, **High**, **Low**, **Close**, **Adjusted Close**, and **Volume**. 

### Key Observations

- **Recent Prices**: 
  - In February 2025, the stock opened at **$332.00** and closed at **$325.83**. 
  - There were fluctuations in prices with highs reaching **$348.04** and lows dropping to **$323.72** during the early February week.

- **Trends Over Time**:
  - The stock experienced significant variation over the year. 
  - A noticeable peak was observed in **December 2024** where the stock reached a high of **$358.07** and closed at **$343.65**, indicating some upward momentum during this period. 
  - The lowest recent price was **$212.00** during **late May 2024**, suggesting a notable drop from earlier highs, with a close price of **$234.44**.

### Price Movements

- **Overall Movement**: 
  - The stock prices have shown a general trend of rising over the long term but with significant drops and rises evident.
  - Frequent fluctuations in the high and low prices indicate demand volatility.
  
- **Volume Analysis**: 
  - The trading volume varied greatly, with the highest volume of **126,360,300** recorded on **May 27, 2024**. High trading volumes often accompany volatility and significant news or events affecting the stock.

### Return Calculation

To calculate the **total returns** over the time period of the data (from the lowest point in May 2024 to the most recent recorded price in February 2025):

1. **Lowest Closing Price**:
   - **May 27, 2024**: Closed at **$234.44**.

2. **Recent Closing Price**:
   - **Feb 7, 2025**: Closed at **$325.83**.

3. **Return Calculation**: 
   \[
   \text{Total Return} = \left( \frac{\text{Final Price} - \text{Initial Price}}{\text{Initial Price}} \right) \times 100
   \]
   \[
   \text{Total Return} = \left( \frac{325.83 - 234.44}{234.44} \right) \times 100 \approx 38.95\%
   \]

### Summary of Trends

- The stock has undergone significant ups and downs throughout the year.
- Despite dips, including a low of **$234.44**, the stock has made a considerable recovery, reaching around **$325.83**, leading to a **total return of approximately 38.95%**.
- Investors looking at CRM's performance should pay attention to periodic price fluctuations and the influences of earnings reports, market conditions, and broader economic factors.

### Conclusion

Salesforce's stock reflects both volatility and recovery potential, appealing to both short-term traders looking for value and long-term investors. A closer look at future earnings announcements and market conditions will be crucial for predicting the next direction of CRM shares.